<a href="https://colab.research.google.com/github/JPSpinola/Actividad-4-Regresi-n-lineal-y-log-stica-/blob/main/Actividad4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importamos las librerias pandas, numpy y matplotlib respectivamente
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
#Cargar archivo csv desde equipo
from google.colab import files
files.upload()

In [3]:
#Carga desde un archivo .csv sin indice
Retailer= pd.read_excel('microretailer_mit_lift_lab_actualizado.xlsx')

In [ ]:
#Verificamos información del Dataframe
Retailer.info()

In [ ]:
#Creamos copia del dataframe
data1=Retailer.copy()
data1

**PREPROCESAMIENTO DE DATOS**

**Valores nulos**

In [ ]:
#Identificar valores nulos por columna
valores_nulos=data1.isnull().sum()
valores_nulos

In [7]:
#Realizamos una copia del dataframe
data2=data1.copy()

In [ ]:
#Utilizamos un metodo con el que sustituimos los valores nulos que esten por detras de valores no nulos por ese mismo valor
data2= data1.fillna(method="bfill")
data2

In [ ]:
#Corroboramos valores nulos
valores_nulos=data2.isnull().sum()
valores_nulos

In [ ]:
#Debido a que todavia nos quedan unos cuantos valores nulos en el dataframe, utilizamos el metodo "ffill" para cambiar los valores nulos que esten delante de un valore no nulo para cambiarlo por el mismo valor
data3= data2.fillna(method='ffill')
data3

In [ ]:
#Corroboramos valores nulos
valores_nulos=data3.isnull().sum()
valores_nulos

In [ ]:
#En caso de que resten unos cuantos valores nulos en el dataframe, utilizamos el metodo de cambiar los valores nulos por un string en concreto para seguridad
data4= data3.fillna('no valido')
data4

**Outliers**

In [13]:
#Creamos otra copia del dataframe
data5=data4.copy()

In [ ]:
#Realizamos un filtro por filas donde se incluyan unicamente los valores cuantitativos
cuantitativas=data5.iloc[:,[13,14,16,21,22,29,31,33,42,49,51,56,57,58,59,60,78,90]] #filas no consecutivas
cuantitativas

In [ ]:
#Realizamos un filtro por filas donde se incluyan unicamente los valores cualitativas
cualitativas=data2.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,15,17,18,19,20,23,24,25,26,27,28,30,32,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,50,52,53,54,55,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,79,80,81,82,83,84,85,86,87,88,89,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105]] #filas no consecutivas
cualitativas

In [ ]:
#Corroboramos valores nulos
valores_nulos=cuantitativas.isnull().sum()
valores_nulos

In [ ]:
#Aplicanmos desviación estandar para encontrar valores extremos
y=cuantitativas
Limite_Superior= y.mean() +3*y.std()
Limite_Inferior= y.mean() -3*y.std()
print("Limite superior permitido", Limite_Superior)
print("Limite inferior permitido", Limite_Inferior)

In [ ]:
#Encontramos Outliers del Dataframe
outliers= cuantitativas[(y>Limite_Superior)|(y<Limite_Inferior)]
outliers

In [ ]:
#Obtenemos datos limpios
data5= cuantitativas[(y<=Limite_Superior)&(y>=Limite_Inferior)]
data5

In [ ]:
#Revisamos valores atípicos (nulos) del dataframe4
valores_nulos=data5.isnull().sum()
valores_nulos

In [ ]:
#Reemplazamos valores atípicos (nulos) del dataframe con "mean"
#Realizamos una copia del dataframe
data_clean=data5.copy()
data_clean=data_clean.fillna(round(data3.mean(),1))
data_clean

In [ ]:
#Corroboramos valores nulos del dataframe LIMPIO
valores_nulos=data_clean.isnull().sum()
valores_nulos

In [ ]:
#Unimos las columnas cuantitativas y cualitativas en un mismo dataframe
Datos_limpios = pd.concat([cualitativas, data_clean], axis=1)
Datos_limpios

In [24]:
#Convertir Dataframe a CSV
Datos_limpios.to_csv("Datos_limpios_Micro_retailer_actualizado.csv")

In [ ]:
#descargar archivo filtrado en csv
from google.colab import files

files.download("Datos_limpios_Micro_retailer_actualizado.csv")

**ACTIVIDAD 4**

**Regresión lineal**

In [ ]:
#Cargar archivo csv desde equipo
from google.colab import files
files.upload()

In [27]:
#Carga desde un archivo .csv sin indice
df= pd.read_csv('Datos_limpios_Micro_retailer_actualizado.csv')

1

In [ ]:
#Se grafican mapas de dispersion de las 3 variables: '97_number_of_customers_in_store', '2_current_permanent_employees' y '145_number_direct_competitors'
from turtle import color

sns.scatterplot(x='103_number_own_fridges', y='268_number_fridges', color='blue', data=df)
sns.scatterplot(x='172_supplier_frequency', y='268_number_fridges', color='red', data=df)
sns.scatterplot(x='145_number_direct_competitors', y='268_number_fridges', color='yellow', data=df)

In [29]:
#Declaramos las variables dependientes e independientes para la regresión lineal.
#Vars_Indep= df[['alcohol', 'speeding', 'no_previous']]
Vars_Indep= df[['103_number_own_fridges', '172_supplier_frequency', '145_number_direct_competitors']]
Var_Dep= df['268_number_fridges']

In [30]:
#Se define model como la función de regresión lineal
from sklearn.linear_model import LinearRegression
model= LinearRegression()

In [ ]:
#Verificamos la funcion relacionada al modelo
type(model)

In [ ]:
#Ajustamos el modelo con las variables antes declaradas
model.fit(X=Vars_Indep, y=Var_Dep)

In [ ]:
#Verificamos los coeficientes obtenidos para el modelo ajustado
model.__dict__

In [ ]:
#Evaluamos la eficiencia del modelo obtenido por medio del coeficiente R^2 Determinación
model.score(Vars_Indep,Var_Dep)

In [ ]:
#Predecimos los valores de total de accidentes a partir de las variables: 'alcohol', 'speeding' y 'no_previous' 
y_pred= model.predict(X=df[['103_number_own_fridges', '172_supplier_frequency', '145_number_direct_competitors']])
y_pred

In [ ]:
#Insertamos la columna de predicciones en el Dataframe
df.insert(0, 'Predicciones', y_pred)
df

In [ ]:
#Visualizamos la gráfica comparativa entre el total real y el total predecido

sns.scatterplot(x= '103_number_own_fridges', y= '268_number_fridges', color= 'blue', data=df)
sns.scatterplot(x= '103_number_own_fridges', y= 'Predicciones', color= 'red', data=df)

In [ ]:
#Visualizamos la gráfica comparativa entre el total real y el total predecido

sns.scatterplot(x= '172_supplier_frequency', y= '268_number_fridges', color= 'blue', data=df)
sns.scatterplot(x= '172_supplier_frequency', y= 'Predicciones', color= 'red', data=df)

In [ ]:
#Visualizamos la gráfica comparativa entre el total real y el total predecido

sns.scatterplot(x= '145_number_direct_competitors', y= '268_number_fridges', color= 'blue', data=df)
sns.scatterplot(x= '145_number_direct_competitors', y= 'Predicciones', color= 'red', data=df)

In [ ]:
#Corroboramos cual es el coeficiente de Determinación de nuestro modelo
coef_Deter=model.score(X=Vars_Indep, y=Var_Dep)
coef_Deter

In [ ]:
#Corroboramos cual es el coeficiente de Correlación de nuestro modelo
coef_Correl=np.sqrt(coef_Deter)
coef_Correl

2

In [ ]:
#Se grafican mapas de dispersion de las 3 variables: '97_number_of_customers_in_store', '2_current_permanent_employees' y '145_number_direct_competitors'
from turtle import color

sns.scatterplot(x='103_number_own_fridges', y='268_number_fridges', color='blue', data=df)
sns.scatterplot(x='172_supplier_frequency', y='268_number_fridges', color='red', data=df)
sns.scatterplot(x='97_number_of_customers_in_store', y='268_number_fridges', color='yellow', data=df)

In [43]:
#Declaramos las variables dependientes e independientes para la regresión lineal.
#Vars_Indep= df[['alcohol', 'speeding', 'no_previous']]
Vars_Indep= df[['103_number_own_fridges', '172_supplier_frequency', '97_number_of_customers_in_store']]
Var_Dep= df['268_number_fridges']

In [44]:
#Se define model como la función de regresión lineal
from sklearn.linear_model import LinearRegression
model= LinearRegression()

In [ ]:
#Verificamos la funcion relacionada al modelo
type(model)

In [ ]:
#Ajustamos el modelo con las variables antes declaradas
model.fit(X=Vars_Indep, y=Var_Dep)

In [ ]:
#Verificamos los coeficientes obtenidos para el modelo ajustado
model.__dict__

In [ ]:
#Evaluamos la eficiencia del modelo obtenido por medio del coeficiente R^2 Determinación
model.score(Vars_Indep,Var_Dep)

In [ ]:
#Predecimos los valores de total de accidentes a partir de las variables: 'alcohol', 'speeding' y 'no_previous' 
y_pred= model.predict(X=df[['103_number_own_fridges', '172_supplier_frequency', '97_number_of_customers_in_store']])
y_pred

In [ ]:
#Insertamos la columna de predicciones en el Dataframe
df.insert(0, 'Predicciones2', y_pred)
df

In [ ]:
#Visualizamos la gráfica comparativa entre el total real y el total predecido

sns.scatterplot(x= '103_number_own_fridges', y= '268_number_fridges', color= 'blue', data=df)
sns.scatterplot(x= '103_number_own_fridges', y= 'Predicciones2', color= 'red', data=df)

In [ ]:
#Visualizamos la gráfica comparativa entre el total real y el total predecido

sns.scatterplot(x= '172_supplier_frequency', y= '268_number_fridges', color= 'blue', data=df)
sns.scatterplot(x= '172_supplier_frequency', y= 'Predicciones2', color= 'red', data=df)

In [ ]:
#Visualizamos la gráfica comparativa entre el total real y el total predecido

sns.scatterplot(x= '97_number_of_customers_in_store', y= '268_number_fridges', color= 'blue', data=df)
sns.scatterplot(x= '97_number_of_customers_in_store', y= 'Predicciones2', color= 'red', data=df)

In [ ]:
#Corroboramos cual es el coeficiente de Determinación de nuestro modelo
coef_Deter=model.score(X=Vars_Indep, y=Var_Dep)
coef_Deter

In [ ]:
#Corroboramos cual es el coeficiente de Correlación de nuestro modelo
coef_Correl=np.sqrt(coef_Deter)
coef_Correl

3

In [ ]:
#Se grafican mapas de dispersion de las 3 variables: '24_burnout', '2_current_permanent_employees' y '145_number_direct_competitors'
from turtle import color

sns.scatterplot(x='103_number_own_fridges', y='268_number_fridges', color='blue', data=df)
sns.scatterplot(x='172_supplier_frequency', y='268_number_fridges', color='red', data=df)
sns.scatterplot(x='24_burnout', y='268_number_fridges', color='yellow', data=df)

In [57]:
#Declaramos las variables dependientes e independientes para la regresión lineal.
#Vars_Indep= df[['alcohol', 'speeding', 'no_previous']]
Vars_Indep= df[['103_number_own_fridges', '172_supplier_frequency', '24_burnout']]
Var_Dep= df['268_number_fridges']

In [58]:
#Se define model como la función de regresión lineal
from sklearn.linear_model import LinearRegression
model= LinearRegression()

In [ ]:
#Verificamos la funcion relacionada al modelo
type(model)

In [ ]:
#Ajustamos el modelo con las variables antes declaradas
model.fit(X=Vars_Indep, y=Var_Dep)

In [ ]:
#Verificamos los coeficientes obtenidos para el modelo ajustado
model.__dict__

In [ ]:
#Evaluamos la eficiencia del modelo obtenido por medio del coeficiente R^2 Determinación
model.score(Vars_Indep,Var_Dep)

In [ ]:
#Predecimos los valores de total de accidentes a partir de las variables: 'alcohol', 'speeding' y 'no_previous' 
y_pred= model.predict(X=df[['103_number_own_fridges', '172_supplier_frequency', '24_burnout']])
y_pred

In [ ]:
#Insertamos la columna de predicciones en el Dataframe
df.insert(0, 'Predicciones3', y_pred)
df

In [ ]:
#Visualizamos la gráfica comparativa entre el total real y el total predecido

sns.scatterplot(x= '103_number_own_fridges', y= '268_number_fridges', color= 'blue', data=df)
sns.scatterplot(x= '103_number_own_fridges', y= 'Predicciones3', color= 'red', data=df)

In [ ]:
#Visualizamos la gráfica comparativa entre el total real y el total predecido

sns.scatterplot(x= '172_supplier_frequency', y= '268_number_fridges', color= 'blue', data=df)
sns.scatterplot(x= '172_supplier_frequency', y= 'Predicciones3', color= 'red', data=df)

In [ ]:
#Visualizamos la gráfica comparativa entre el total real y el total predecido

sns.scatterplot(x= '24_burnout', y= '268_number_fridges', color= 'blue', data=df)
sns.scatterplot(x= '24_burnout', y= 'Predicciones3', color= 'red', data=df)

In [ ]:
#Corroboramos cual es el coeficiente de Determinación de nuestro modelo
coef_Deter=model.score(X=Vars_Indep, y=Var_Dep)
coef_Deter

In [ ]:
#Corroboramos cual es el coeficiente de Correlación de nuestro modelo
coef_Correl=np.sqrt(coef_Deter)
coef_Correl

**Regresión logística**

In [ ]:
#Cargar archivo csv desde equipo
from google.colab import files
files.upload()

In [ ]:
#Carga desde un archivo .csv sin indice
df= pd.read_csv('Datos_limpios_Micro_retailer_actualizado.csv')
df.head(5)

In [72]:
#Rellenamos nulos dentro del condigo 
df=df.fillna(method='ffill')

1

In [73]:
#Declaramos las variables dependientes e independientes para la regresión logística.
Vars_Indep= df[['97_number_of_customers_in_store', '317_home_deliveries', '2_current_permanent_employees']]
Var_Dep= df['186_internet_connection']

In [74]:
#Redefinimos las variables
X= Vars_Indep 
y= Var_Dep

In [75]:
#Dividimos el conjunto de datos en la parte de entrenamiento y pruba:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

In [76]:
#Se escalan todos los datos 
escalar= StandardScaler()

In [77]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba 
X_train=escalar.fit_transform(X_train)
X_test= escalar.transform(X_test)

In [78]:
#Definimos el algoritmo a utilizar 
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

In [ ]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

In [ ]:
#Verifico matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

In [ ]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average='binary', pos_label="yes")
print('Precision del modelo:')
print(precision)

In [ ]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

In [ ]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="yes")
print('Sensibilidad del modelo:')
print(sensibilidad)

2

In [85]:
#Declaramos las variables dependientes e independientes para la regresión logística.
Vars_Indep= df[['104_how_many_shelves_does_the_micro_retailer_have', '172_supplier_frequency', '103_number_own_fridges']]
Var_Dep= df['20_reviews_finances_monthly']

In [86]:
#Redefinimos las variables
X= Vars_Indep 
y= Var_Dep

In [87]:
#Dividimos el conjunto de datos en la parte de entrenamiento y pruba:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

In [88]:
#Se escalan todos los datos 
escalar= StandardScaler()

In [89]:
#Para realizar el escalamiento de las variables "A" tanto de entrenamiento como de prueba 
X_train=escalar.fit_transform(X_train)
X_test= escalar.transform(X_test)

In [90]:
#Definimos el algoritmo a utilizar 
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

In [ ]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

In [ ]:
#Verifico matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

In [ ]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average='binary', pos_label="yes")
print('Precision del modelo:')
print(precision)

In [ ]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

In [ ]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="yes")
print('Sensibilidad del modelo:')
print(sensibilidad)

3

In [97]:
#Declaramos las variables dependientes e independientes para la regresión logística.
Vars_Indep= df[['145_number_direct_competitors', '315_frequency_topups', '268_number_fridges']]
Var_Dep= df['310_burnout']

In [98]:
#Redefinimos las variables
X= Vars_Indep 
y= Var_Dep

In [99]:
#Dividimos el conjunto de datos en la parte de entrenamiento y pruba:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

In [100]:
#Se escalan todos los datos 
escalar= StandardScaler()

In [101]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba 
X_train=escalar.fit_transform(X_train)
X_test= escalar.transform(X_test)

In [102]:
#Definimos el algoritmo a utilizar 
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()

In [ ]:
#Entrenamos el modelo 
algoritmo.fit(X_train, y_train)

In [ ]:
#Realiamos una prediccion 
y_pred=algoritmo.predict(X_test)
#y_pred_df = pd.DataFrame(y_pred)
y_pred

In [ ]:
#Verifico matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

In [ ]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average='binary', pos_label="yes")
print('Precision del modelo:')
print(precision)

In [ ]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

In [ ]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="yes")
print('Sensibilidad del modelo:')
print(sensibilidad)